## 1. 필요 라이브러리 및 패키지 import, Data Search
----------
Baseline에서는 CountVectorizer를 사용함

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

### Data Load
- 추가로 주어지는 Python Code 데이터들을 활용하여 새로운 Pair 쌍을 생성하여 더 많은 데이터로 학습 가능
- 베이스라인에서는 이미 Pair 쌍으로 구축되어 주어지는 sample 데이터들로 학습

In [2]:
train = pd.read_csv("data/sample_train.csv")
train.head(10)

,code1,code2,similar
0,"flag = ""go""\ncnt = 0\nwhile flag == ""go"":\n ...",# Python 3+\n#--------------------------------...,1
1,"b, c = map(int, input().split())\n\nprint(b * c)",import numpy as np\n\nn = int(input())\na = np...,0
2,import numpy as np\nimport sys\nread = sys.std...,"N, M = map(int, input().split())\nif M%2 != 0:...",0
3,"b, c = map(int, input().split())\n\nprint(b * c)","n,m=map(int,input().split())\nh=list(map(int,i...",0
4,s=input()\nt=input()\nans=0\nfor i in range(le...,"import math\na,b,h,m=map(int,input().split())\...",0
5,"n,m = map(int,input().split())\nl=1\nif n%2==1...","N = int(input())\nL = list(map(int, input().sp...",0
6,"N = int(input())\nP = list(map(int, input().sp...",input()\n\nnumbers = input().split()\n\nnumber...,0
7,"N = int(input())\nA = list(map(int, input().sp...","sum = 1\ninput()\nfor i in map(int,input().spl...",1
8,"from sys import stdin, setrecursionlimit\n\nWE...",from statistics import median\n#import collect...,1
9,"n,k = map(int,input().split())\n\na = (n-k+2)*...",def make_divisors(n):\n divisors = []\n ...,0


In [3]:
test = pd.read_csv("data/test.csv")
test.head()

,pair_id,code1,code2
0,1,def main():\n s = input()\n if s.count('a') ...,"N,K = map(int,input().split())\nA = list(map(i..."
1,2,"N,K,Q = map(int,input().split())\npoints = [0]...","N, K, Q = map(int,input().split())\n\nif K > Q..."
2,3,from itertools import combinations\nn = int(in...,s = input()\nt = input()\nlength_s = len(s)\nl...
3,4,"a,b=map(int,input().split())\n\nans1=a+b\nans2...","a, b, c, d = map(int,input().split())\n\nif a ..."
4,5,S = input()\nK = int(input())\n\nind = -1\nfor...,"H, W = map(int, input().split())\ngrid = []\nf..."


## 2. Define Model & Train
------
- CountVectorizer와 CosineSimilarity 사용
- Baseline 코드에서는 정말 기본적인 vectorizer로만 학습을 시킴

In [4]:
class BaselineModel():
    def __init__(self, threshold=0.5):
        super(BaselineModel, self).__init__()
        self.threshold = threshold # 유사도 임계값
        self.vectorizer = CountVectorizer()
        
    def fit(self, code1, code2):
        # 입력 받은 코드 쌍으로 부터 vectorizer를 fit 시킵니다.
        self.vectorizer.fit(code1)
        self.vectorizer.fit(code2)
        print('Done.')
    
    def predict_proba(self, code1, code2):
        # 입력 받은 코드 쌍으로 부터 vectorizer를 통해 vector화 합니다.
        code1_vecs = self.vectorizer.transform(code1)
        code2_vecs = self.vectorizer.transform(code2)
        
        preds = []
        # 각각의 코드 쌍(=벡터 쌍)으로부터 cosine-similarity를 구합니다.
        for code1_vec, code2_vec in zip(code1_vecs, code2_vecs):
            preds.append(cosine_similarity(code1_vec, code2_vec))
        
        preds = np.reshape(preds, len(preds))
        print('Done.')
        # 각 코드 쌍들의 유사도를 반환
        return preds
    
    def predict(self, code1, code2):
        preds = self.predict_proba(code1, code2)
        # cosine-similarity (유사도)가 설정한 임계값(Threshold=0.5)보다 높다면 유사하다 : 1, 아니라면 유사하지 않다 : 0
        preds = np.where(preds>self.threshold, 1, 0)
        # 각 코드 쌍들의 유사도를 Threshold를 통해 유사함을 판별 (이진분류)
        return preds

In [5]:
# 모델 선언
model = BaselineModel(threshold=0.5)
# 학습 코드 쌍들로부터 Model을 Fitting
model.fit(train['code1'], train['code2'])

Done.


In [6]:
# 모델 추론
preds = model.predict(test['code1'], test['code2'])

Done.


In [7]:
submission = pd.read_csv('data/sample_submission.csv')
submission['similar'] = preds
submission.to_csv('result/submission_baseline.csv', index=False)